In [ ]:
# Monitors library source files and recompiles them after most changes
import Revise

# Run the init script which will setup the JDP project if necessary
include("../src/init.jl")

using Markdown
import Dates: Day

# Import some libraries from the JDP project
using JDP.BugRefs
using JDP.Tracker
using JDP.Spammer
using JDP.Trackers.OpenQA    # Contains functions for dealing with the OpenQA web API
using JDP.Repository
using JDP.Functional

trackers = load_trackers();

args = try
    WEAVE_ARGS
catch
    Dict()
end

In [ ]:
product = get(args, "product", "opensuse-Tumbleweed")
dryrun = get(args, "dryrun", false);
host = get(args, "host", "ooo")

# Bug Tag Propagation

This partially automates copying existing bug tags from one failed test result to another.

## Find untagged test failures

First we fetch the test results from OpenQA.

In [ ]:
# Get some cached job results from the instance specified by `host`
# The final arg limits the number of results which will be loaded
allres = Repository.fetch(OpenQA.TestResult, Vector, host, OpenQA.RecentOrInterestingJobsDef)
md"We now have $(length(allres)) test results"

Next we find the latest build for product which matches our product filter.

In [ ]:
prodbuilds = OpenQA.get_product_builds(filter(r -> occursin(Regex(product), r.product), allres))

[p => [b.orig for b in Iterators.take(bs, 5)] for (p, bs) in prodbuilds]

Then filter the results to only include fails from a particular build with no bug references.

In [ ]:
untagged = filter(allres) do res
    (builds = get(prodbuilds, res.product, nothing)) ≠ nothing &&
    first(builds).orig == res.build &&
    occursin(r"failed", res.result) &&
    (isempty(res.refs) || all(rf -> rf.negated, res.refs))
end

for res in untagged
    display(res)
end

## Find existing bug references for tests

Now we search all the test results for bug tags that were added in other builds or architectures.

In [ ]:
fqn = OpenQA.get_fqn

tagdict = Dict{String, Set{JDP.BugRefs.Ref}}(fqn(res) => Set() for res in untagged)

for res in allres
    if haskey(tagdict, fqn(res)) && length(res.refs) > 0
        refs = tagdict[fqn(res)]
        
        for rf in res.refs
            # Negated bugrefs permanently stop a bugref from being used on the same test
            if rf.negated
                delete!(refs, BugRefs.Ref(rf.tracker, rf.id, false))
                push!(refs, rf)
            elseif !(BugRefs.Ref(rf.tracker, rf.id, true) in refs)
                push!(refs, rf)
            end
        end
    end
end

# Anti-tags are not propogated for the same reason graveyards don't scale
for refs in values(tagdict)
    filter!(rf -> !rf.negated, refs)
end

length(tagdict)

In [ ]:
filter(p -> !isempty(p[2]), pairs(tagdict))

Optionally we can get a summary of the bug tags it has found from Bugzilla, Redmine and any other trackers which we have defined a `Bug` type for.

In [ ]:
bugdict = Dict(name => [] for name in keys(tagdict))
for (k, v) in pairs(tagdict)
    for rf in v
        bug = Repository.fetch(rf)
        push!(bugdict[k], rf => bug)
    end
end

bugdict

In [ ]:
mdbuf = IOBuffer()

for (k, v) in pairs(bugdict)
    if isempty(v)
        continue
    end
    
    println(mdbuf, "- ", k)
    for (rf, bug) in v
        print(mdbuf, "   * ")
        show(mdbuf, MIME("text/markdown"), rf)
        print(mdbuf, " ")
        if bug != nothing
            show(mdbuf, MIME("text/markdown"), bug)
        else
            print(mdbuf, "*no data*")
        end
        println(mdbuf)
    end
    
end

seek(mdbuf, 0)
Markdown.parse(mdbuf)

## Manual additions and removals

If necessary some tags can be removed, although you should generally use anti-tags or modify the search algorithm.

In [ ]:
modded = []

for (t, rf) in []
    if haskey(tagdict, t)
        delete!(tagdict[t], BugRefs.Ref(rf, trackers))
        push!(modded, t => tagdict[t])
    end
end

modded

While others can be added, which may be quicker than commenting directly on OpenQA or implementing some new tagging logic.

In [ ]:
modded = []

for (t, rf) in []
    if haskey(tagdict, t)
        delete!(tagdict[t], BugRefs.Ref(rf[1], trackers, !rf[2]))
        push!(tagdict[t], BugRefs.Ref(rf[1], trackers, rf[2]))
        push!(modded, t => tagdict[t])
    end
end

modded

## Posting back to OpenQA

Now we can post the results back to OpenQA. First we create a dictionary of jobs which we will post the bug tags to.

In [ ]:
taggings = Dict()

for res in untagged
    for rf in tagdict[fqn(res)]
        tags = get!(taggings, res.job.id, [])
        push!(tags, res => rf)
    end
end
        
taggings     

In [ ]:
oqa = get_tracker(trackers, host)
ses = Tracker.ensure_login!(oqa)
modified_jobs = Set{Int64}()

if dryrun
    @warn "Nothing will be posted because dryrun is set"
end

for (jid, refs) in taggings
    mdbuf = IOBuffer()
    
    print(mdbuf,
        "This is an automated message from the [JDP Propagate Bug Tags](https://rpalethorpe.io.suse.de/jdp/reports/Propagate%20Bug%20Tags.html) report",
        "<br><br>",
        "The following bug tags have been propagated: <br>")
    for (res, rf) in refs
        print(mdbuf, res.name, rf.negated ? ":" : ": ", rf)
        if rf.negated
            print(mdbuf, " *This is an anti-tag to prevent the following bug being used again* <br>")
        end
        print(mdbuf, " [")
        show(mdbuf, MIME("text/markdown"), Repository.fetch(rf))
        print(mdbuf, "] <br>")
    end
    
    text = String(take!(mdbuf))
    
    @info "Posting comment to job $jid" text
    display(Markdown.parse(text))
    if !dryrun
        OpenQA.post_job_comment(ses, jid, text)
        push!(modified_jobs, jid)
    end
end

## Notifications

Send out notifications if some tags were propagated.

In [ ]:
if !isempty(taggings) || !isempty(untagged)
    spam = IOBuffer()
    
    if !isempty(taggings)
        println(spam, length(taggings), " bug tags propagated on:")
        for (p, bs) in prodbuilds
            println(spam, "  * ", p, " build ", first(bs).orig)
        end
    end
    
    ulen = sum(untagged) do res
        haskey(taggings, res.name) ? 0 : 1
    end
    if ulen > 0
        flag_name = "notify-untagged"
        flag_val = join((first(bs).orig for bs in values(prodbuilds)), "-")
        if flag_val ≠ Repository.get_temp_flag(flag_name)
            println(spam, ulen, " failing tests are still missing bug tags!")
            Repository.set_temp_flag(flag_name, flag_val, Day(2))
        end
    end
    
    if position(spam) > 0
        println(spam, "See the [Propagate Bug Tags](https://rpalethorpe.io.suse.de/jdp/reports/Propagate%20Bug%20Tags.html) report for details.")
    end    
    text = String(take!(spam))
    
    if dryrun
        display(Markdown.parse(text))
    elseif !isempty(text)
        Spammer.post_message(Spammer.Message(text, ["rpalethorpe"]))
    end
end

## Refresh effected jobs

Redownload the comments for jobs which we just tried to post to.

In [ ]:
let n = length(modified_jobs)
    @info "$host: Refreshing $n job comments"

    for (i, jid) in enumerate(modified_jobs)
        @info "$host: GET job $jid ($i of $n)"
        
        comments = try
            OpenQA.json_to_comments(OpenQA.get_job_comments(ses, jid))
        catch
            nothing
        end
        
        if comments != nothing
            job = Repository.fetch(OpenQA.JobResult, host, jid)
            job.comments = comments
            Repository.store("$host-job-$(jid)", job)
        end
    end
end